<a href="https://colab.research.google.com/github/SUPERREALCODER/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from psutil import virtual_memory
ram = virtual_memory().total / 1e9
print(f'Total RAM: {ram} GB')


Total RAM: 13.609455616 GB


In [1]:
!pip3 install llmware

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 29.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvi

In [2]:

"""     Fast Start Example #3 - Prompts & Model Catalog - how to build prompts and run inferences

    In this example, we will illustrate:

    1.  Discovery - how to discover models in the llmware ModelCatalog
    2.  Load Model - how to load a selected model from the catalog
    3.  Prompt - how to create a basic prompt and run an inference with the model

"""


import time
from llmware.prompts import Prompt
from llmware.models import ModelCatalog


def hello_world_questions():

    """ This is a set of useful test questions to do a 'hello world' but there is nothing special about the
    questions - please feel free to edit and ask your own queries with your own context passages.

    --if you are using one of the llmware models, please take note that the models have been trained to answer
    based on the information provided, so if you ask a question without passing any context passage, then
    don't be surprised if the model responds with 'Not Found.' """

    test_list = [

    {"query": "What is the total amount of the invoice?",
     "answer": "$22,500.00",
     "context": "Services Vendor Inc. \n100 Elm Street Pleasantville, NY \nTO Alpha Inc. 5900 1st Street "
                "Los Angeles, CA \nDescription Front End Engineering Service $5000.00 \n Back End Engineering"
                " Service $7500.00 \n Quality Assurance Manager $10,000.00 \n Total Amount $22,500.00 \n"
                "Make all checks payable to Services Vendor Inc. Payment is due within 30 days."
                "If you have any questions concerning this invoice, contact Bia Hermes. "
                "THANK YOU FOR YOUR BUSINESS!  INVOICE INVOICE # 0001 DATE 01/01/2022 FOR Alpha Project P.O. # 1000"},

    {"query": "What was the amount of the trade surplus?",
     "answer": "62.4 billion yen ($416.6 million)",
     "context": "Japan’s September trade balance swings into surplus, surprising expectations"
                "Japan recorded a trade surplus of 62.4 billion yen ($416.6 million) for September, "
                "beating expectations from economists polled by Reuters for a trade deficit of 42.5 "
                "billion yen. Data from Japan’s customs agency revealed that exports in September "
                "increased 4.3% year on year, while imports slid 16.3% compared to the same period "
                "last year. According to FactSet, exports to Asia fell for the ninth straight month, "
                "which reflected ongoing China weakness. Exports were supported by shipments to "
                "Western markets, FactSet added. — Lim Hui Jie"},

    {"query": "What was Microsoft's revenue in the 3rd quarter?",
     "answer": "$52.9 billion",
     "context": "Microsoft Cloud Strength Drives Third Quarter Results \nREDMOND, Wash. — April 25, 2023 — "
                "Microsoft Corp. today announced the following results for the quarter ended March 31, 2023,"
                " as compared to the corresponding period of last fiscal year:\n· Revenue was $52.9 billion"
                " and increased 7% (up 10% in constant currency)\n· Operating income was $22.4 billion "
                "and increased 10% (up 15% in constant currency)\n· Net income was $18.3 billion and "
                "increased 9% (up 14% in constant currency)\n· Diluted earnings per share was $2.45 "
                "and increased 10% (up 14% in constant currency).\n"},

    {"query": "When did the LISP machine market collapse?",
     "answer": "1987.",
     "context": "The attendees became the leaders of AI research in the 1960s."
                "  They and their students produced programs that the press described as 'astonishing': "
                "computers were learning checkers strategies, solving word problems in algebra, "
                "proving logical theorems and speaking English.  By the middle of the 1960s, research in "
                "the U.S. was heavily funded by the Department of Defense and laboratories had been "
                "established around the world. Herbert Simon predicted, 'machines will be capable, "
                "within twenty years, of doing any work a man can do'.  Marvin Minsky agreed, writing, "
                "'within a generation ... the problem of creating 'artificial intelligence' will "
                "substantially be solved'. They had, however, underestimated the difficulty of the problem.  "
                "Both the U.S. and British governments cut off exploratory research in response "
                "to the criticism of Sir James Lighthill and ongoing pressure from the US Congress "
                "to fund more productive projects. Minsky's and Papert's book Perceptrons was understood "
                "as proving that artificial neural networks approach would never be useful for solving "
                "real-world tasks, thus discrediting the approach altogether.  The 'AI winter', a period "
                "when obtaining funding for AI projects was difficult, followed.  In the early 1980s, "
                "AI research was revived by the commercial success of expert systems, a form of AI "
                "program that simulated the knowledge and analytical skills of human experts. By 1985, "
                "the market for AI had reached over a billion dollars. At the same time, Japan's fifth "
                "generation computer project inspired the U.S. and British governments to restore funding "
                "for academic research. However, beginning with the collapse of the Lisp Machine market "
                "in 1987, AI once again fell into disrepute, and a second, longer-lasting winter began."},

    {"query": "When will employment start?",
     "answer": "April 16, 2012.",
     "context": "THIS EXECUTIVE EMPLOYMENT AGREEMENT (this “Agreement”) is entered "
                "into this 2nd day of April, 2012, by and between Aphrodite Apollo "
                "(“Executive”) and TestCo Software, Inc. (the “Company” or “Employer”), "
                "and shall become effective upon Executive’s commencement of employment "
                "(the “Effective Date”) which is expected to commence on April 16, 2012. "
                "The Company and Executive agree that unless Executive has commenced "
                "employment with the Company as of April 16, 2012 (or such later date as "
                "agreed by each of the Company and Executive) this Agreement shall be "
                "null and void and of no further effect."},

    {"query": "What is the current rate on 10-year treasuries?",
     "answer": "4.58%",
     "context": "Stocks rallied Friday even after the release of stronger-than-expected U.S. jobs data "
                "and a major increase in Treasury yields.  The Dow Jones Industrial Average gained 195.12 points, "
                "or 0.76%, to close at 31,419.58. The S&P 500 added 1.59% at 4,008.50. The tech-heavy "
                "Nasdaq Composite rose 1.35%, closing at 12,299.68. The U.S. economy added 438,000 jobs in "
                "August, the Labor Department said. Economists polled by Dow Jones expected 273,000 "
                "jobs. However, wages rose less than expected last month.  Stocks posted a stunning "
                "turnaround on Friday, after initially falling on the stronger-than-expected jobs report. "
                "At its session low, the Dow had fallen as much as 198 points; it surged by more than "
                "500 points at the height of the rally. The Nasdaq and the S&P 500 slid by 0.8% during "
                "their lowest points in the day.  Traders were unclear of the reason for the intraday "
                "reversal. Some noted it could be the softer wage number in the jobs report that made "
                "investors rethink their earlier bearish stance. Others noted the pullback in yields from "
                "the day’s highs. Part of the rally may just be to do a market that had gotten extremely "
                "oversold with the S&P 500 at one point this week down more than 9% from its high earlier "
                "this year.  Yields initially surged after the report, with the 10-year Treasury rate trading "
                "near its highest level in 14 years. The benchmark rate later eased from those levels, but "
                "was still up around 6 basis points at 4.58%.  'We’re seeing a little bit of a give back "
                "in yields from where we were around 4.8%. [With] them pulling back a bit, I think that’s "
                "helping the stock market,' said Margaret Jones, chief investment officer at Vibrant Industries "
                "Capital Advisors. 'We’ve had a lot of weakness in the market in recent weeks, and potentially "
                "some oversold conditions.'"},

    {"query": "What is the governing law?",
     "answer": "State of Massachusetts",
     "context": "19.	Governing Law and Procedures. This Agreement shall be governed by and interpreted "
                 "under the laws of the State of Massachusetts, except with respect to Section 18(a) of this Agreement,"
                 " which shall be governed by the laws of the State of Delaware, without giving effect to any "
                 "conflict of laws provisions. Employer and Executive each irrevocably and unconditionally "
                 "(a) agrees that any action commenced by Employer for preliminary and permanent injunctive relief "
                 "or other equitable relief related to this Agreement or any action commenced by Executive pursuant "
                 "to any provision hereof, may be brought in the United States District Court for the federal "
                 "district in which Executive’s principal place of employment is located, or if such court does "
                 "not have jurisdiction or will not accept jurisdiction, in any court of general jurisdiction "
                 "in the state and county in which Executive’s principal place of employment is located, "
                 "(b) consents to the non-exclusive jurisdiction of any such court in any such suit, action o"
                 "r proceeding, and (c) waives any objection which Employer or Executive may have to the "
                 "laying of venue of any such suit, action or proceeding in any such court. Employer and "
                 "Executive each also irrevocably and unconditionally consents to the service of any process, "
                 "pleadings, notices or other papers in a manner permitted by the notice provisions of Section 8."},

    {"query": "What is the amount of the base salary?",
     "answer": "$200,000.",
     "context": "2.2. Base Salary. For all the services rendered by Executive hereunder, during the "
                 "Employment Period, Employer shall pay Executive a base salary at the annual rate of "
                 "$200,000, payable semimonthly in accordance with Employer’s normal payroll practices. "
                 "Executive’s base salary shall be reviewed annually by the Board (or the compensation committee "
                 "of the Board), pursuant to Employer’s normal compensation and performance review policies "
                 "for senior level executives, and may be increased but not decreased. The amount of any "
                 "increase for each year shall be determined accordingly. For purposes of this Agreement, "
                 "the term “Base Salary” shall mean the amount of Executive’s base salary established "
                 "from time to time pursuant to this Section 2.2. "},

    {"query": "Is the expected gross margin greater than 70%?",
     "answer": "Yes, between 71.5% and 72.%",
     "context": "Outlook NVIDIA’s outlook for the third quarter of fiscal 2024 is as follows:"
                "Revenue is expected to be $16.00 billion, plus or minus 2%. GAAP and non-GAAP "
                "gross margins are expected to be 71.5% and 72.5%, respectively, plus or minus "
                "50 basis points.  GAAP and non-GAAP operating expenses are expected to be "
                "approximately $2.95 billion and $2.00 billion, respectively.  GAAP and non-GAAP "
                "other income and expense are expected to be an income of approximately $100 "
                "million, excluding gains and losses from non-affiliated investments. GAAP and "
                "non-GAAP tax rates are expected to be 14.5%, plus or minus 1%, excluding any discrete items."
                "Highlights NVIDIA achieved progress since its previous earnings announcement "
                "in these areas:  Data Center Second-quarter revenue was a record $10.32 billion, "
                "up 141% from the previous quarter and up 171% from a year ago. Announced that the "
                "NVIDIA® GH200 Grace™ Hopper™ Superchip for complex AI and HPC workloads is shipping "
                "this quarter, with a second-generation version with HBM3e memory expected to ship "
                "in Q2 of calendar 2024. "},

    {"query": "What is Bank of America's rating on Target?",
     "answer": "Buy",
     "context": "Here are some of the tickers on my radar for Thursday, Oct. 12, taken directly from "
                "my reporter’s notebook: It’s the one-year anniversary of the S&P 500′s bear market bottom "
                "of 3,577. Since then, as of Wednesday’s close of 4,376, the broad market index "
                "soared more than 22%.  Hotter than expected September consumer price index, consumer "
                "inflation. The Social Security Administration issues announced a 3.2% cost-of-living "
                "adjustment for 2024.  Chipotle Mexican Grill (CMG) plans price increases. Pricing power. "
                "Cites consumer price index showing sticky retail inflation for the fourth time "
                "in two years. Bank of America upgrades Target (TGT) to buy from neutral. Cites "
                "risk/reward from depressed levels. Traffic could improve. Gross margin upside. "
                "Merchandising better. Freight and transportation better. Target to report quarter "
                "next month. In retail, the CNBC Investing Club portfolio owns TJX Companies (TJX), "
                "the off-price juggernaut behind T.J. Maxx, Marshalls and HomeGoods. Goldman Sachs "
                "tactical buy trades on Club names Wells Fargo (WFC), which reports quarter Friday, "
                "Humana (HUM) and Nvidia (NVDA). BofA initiates Snowflake (SNOW) with a buy rating."
                "If you like this story, sign up for Jim Cramer’s Top 10 Morning Thoughts on the "
                "Market email newsletter for free. Barclays cuts price targets on consumer products: "
                "UTZ Brands (UTZ) to $16 per share from $17. Kraft Heinz (KHC) to $36 per share from "
                "$38. Cyclical drag. J.M. Smucker (SJM) to $129 from $160. Secular headwinds. "
                "Coca-Cola (KO) to $59 from $70. Barclays cut PTs on housing-related stocks: Toll Brothers"
                "(TOL) to $74 per share from $82. Keeps underweight. Lowers Trex (TREX) and Azek"
                "(AZEK), too. Goldman Sachs (GS) announces sale of fintech platform and warns on "
                "third quarter of 19-cent per share drag on earnings. The buyer: investors led by "
                "private equity firm Sixth Street. Exiting a mistake. Rise in consumer engagement for "
                "Spotify (SPOT), says Morgan Stanley. The analysts hike price target to $190 per share "
                "from $185. Keeps overweight (buy) rating. JPMorgan loves elf Beauty (ELF). Keeps "
                "overweight (buy) rating but lowers price target to $139 per share from $150. "
                "Sees “still challenging” environment into third-quarter print. The Club owns shares "
                "in high-end beauty company Estee Lauder (EL). Barclays upgrades First Solar (FSLR) "
                "to overweight from equal weight (buy from hold) but lowers price target to $224 per "
                "share from $230. Risk reward upgrade. Best visibility of utility scale names."},

    {"query": "Who is NVIDIA's partner for the driver assistance system?",
     "answer": "MediaTek",
     "context":   "Automotive Second-quarter revenue was $253 million, down 15% from the previous "
                  "quarter and up 15% from a year ago. Announced that NVIDIA DRIVE Orin™ is powering "
                  "the new XPENG G6 Coupe SUV’s intelligent advanced driver assistance system. "
                  "Partnered with MediaTek, which will develop mainstream automotive systems on "
                  "chips for global OEMs, which integrate new NVIDIA GPU chiplet IP for AI and graphics."},

    {"query": "What was the rate of decline in 3rd quarter sales?",
     "answer": "20% year-on-year.",
     "context": "Nokia said it would cut up to 14,000 jobs as part of a cost cutting plan following "
                "third quarter earnings that plunged. The Finnish telecommunications giant said that "
                "it will reduce its cost base and increase operation efficiency to “address the "
                "challenging market environment. The substantial layoffs come after Nokia reported "
                "third-quarter net sales declined 20% year-on-year to 4.98 billion euros. Profit over "
                "the period plunged by 69% year-on-year to 133 million euros."},

    {"query": "What was professional visualization revenue in the quarter?",
     "answer": "$379 million",
     "context": "Gaming Second-quarter revenue was $2.49 billion, up 11% from the previous quarter and up "
                "22% from a year ago. Began shipping the GeForce RTX™ 4060 family of GPUs, "
                "bringing to gamers NVIDIA Ada Lovelace architecture and DLSS, starting at $299."
                "Announced NVIDIA Avatar Cloud Engine, or ACE, for Games, a custom AI model "
                "foundry service using AI-powered natural language interactions to transform games "
                "by bringing intelligence to non-playable characters. Added 35 DLSS games, including "
                "Diablo IV, Ratchet & Clank: Rift Apart, Baldur’s Gate 3 and F1 23, as well as Portal: "
                "Prelude RTX, a path-traced game made by the community using NVIDIA’s RTX Remix creator tool."
                "Professional Visualization Second-quarter revenue was $379 million, up 28% from the "
                "previous quarter and down 24% from a year ago.  Announced three new desktop "
                "workstation RTX GPUs based on the Ada Lovelace architecture — NVIDIA RTX 5000, RTX 4500 "
                "and RTX 4000 — to deliver the latest AI, graphics and real-time rendering, which are "
                "shipping this quarter. Announced a major release of the NVIDIA Omniverse platform, "
                "with new foundation applications and services for developers and industrial "
                "enterprises to optimize and enhance their 3D pipelines with OpenUSD and "
                "generative AI.  Joined with Pixar, Adobe, Apple and Autodesk to form the "
                "Alliance for OpenUSD to promote the standardization, development, evolution and "
                "growth of Universal Scene Description technology."},


    {"query": "What is the executive's title?",
     "answer": "Senior Vice President, Event Planning ('SVP') of the Workforce Optimization Division.",
     "context": "2.1. Duties and Responsibilities and Extent of Service. During the Employment Period, "
                 "Executive shall serve as Senior Vice President, Event Planning (“SVP”) of the Employer’s "
                 "Workforce Optimization Division. In such role, Executive will report to the Board of "
                 "Directors of Employer (the “Board”) and shall devote substantially all of his business time "
                 "and attention and his best efforts and ability to the operations of Employer and its subsidiaries. "
                 "Executive shall be responsible for running Employer’s day-to-day operations and shall perform "
                 "faithfully, diligently and competently the duties and responsibilities of a SVP and such other "
                 "duties and responsibilities as directed by the Board and are consistent with such position. "
                 "The foregoing shall not be construed as preventing Executive from (a) making passive "
                 "investments in other businesses or enterprises consistent with Employer’s code of conduct, "
                 "or (b) engaging in any other business activity consistent with Employer’s code of conduct; "
                 "provided that Executive seeks and obtains the prior approval of the Board before engaging "
                 "in any other business activity. In addition, it shall not be a violation of this Agreement "
                 "for Executive to participate in civic or charitable activities, deliver lectures, fulfill "
                 "speaking engagements, teach at educational institutions, and/or manage personal investments "
                 "(subject to the immediately preceding sentence); provided that such activities do not "
                 "interfere in any substantial respect with the performance of Executive’s responsibilities "
                 "as an employee in accordance with this Agreement. Executive may also serve on one or more "
                 "corporate boards of another company (and committees thereof) upon giving advance notice "
                 "to the Board prior to commencing service on any other corporate board."},

        {"query": "According to the CFO, what led to the increase in cloud revenue?",
         "answer": "Focused execution by our sales teams and partners",
         "context": "'The world's most advanced AI models "
                  "are coming together with the world's most universal user interface - natural language - "
                  "to create a new era of computing,' said Satya Nadella, chairman and chief "
                  "executive officer of Microsoft. 'Across the Microsoft Cloud, we are the platform "
                  "of choice to help customers get the most value out of their digital spend and innovate "
                  "for this next generation of AI.' 'Focused execution by our sales teams and partners "
                  "in this dynamic environment resulted in Microsoft Cloud revenue of $28.5 billion, "
                  "up 22% (up 25% in constant currency) year-over-year,' said Amy Hood, executive "
                  "vice president and chief financial officer of Microsoft.\n"},

    {"query": "Which company is located in Nevada?",
     "answer": "North Industries",
     "context": "To send notices to Blue Moon Tech, mail to their headquarters at: "
                "555 California Street, San Francisco, California 94123. To send notices to North Industries, mail to"
                "their principal U.S. offices at: 19832 32nd Avenue, Las Vegas, Nevada 23593.\nTo send notices "
                "to Red River Industries, send to: One Red River Road, Stamford, Connecticut 08234."},

    {"query": "When can termination after a material breach occur?",
     "answer": "If the breach is not cured within 15 days of notice of the breach.",
     "context": "This Agreement shall remain in effect until terminated. Either party may terminate this "
             "agreement, any Statement of Work or Services Description for convenience by giving the other "
             "party 30 days written notice. Either party may terminate this Agreement or any work order or "
             "services description if the other party is in material breach or default of any obligation "
             "that is not cured within 15 days’ notice of such breach. The TestCo agrees to pay all fees "
             "for services performed and expenses incurred prior to the termination of this Agreement. "
             "Termination of this Agreement will terminate all outstanding Statement of Work or Services "
             "Description entered into under this agreement."},

    {"query": "What is a headline summary in 10 words or less?",
     "answer": "Joe Biden is the 46th President of the United States.",
     "context": "Joe Biden's tenure as the 46th president of the United States began with "
                "his inauguration on January 20, 2021. Biden, a Democrat from Delaware who "
                "previously served as vice president under Barack Obama, "
                "took office following his victory in the 2020 presidential election over "
                "Republican incumbent president Donald Trump. Upon his inauguration, he "
                "became the oldest president in American history."},

    {"query": "Who are the two people that won elections in Georgia?",
     "answer": "Jon Ossoff and Raphael Warnock",
     "context": "Though Biden was generally acknowledged as the winner, "
                "General Services Administration head Emily W. Murphy "
                 "initially refused to begin the transition to the president-elect, "
                 "thereby denying funds and office space to his team. "
                 "On November 23, after Michigan certified its results, Murphy "
                 "issued the letter of ascertainment, granting the Biden transition "
                 "team access to federal funds and resources for an orderly transition. "
                 "Two days after becoming the projected winner of the 2020 election, "
                 "Biden announced the formation of a task force to advise him on the "
                 "COVID-19 pandemic during the transition, co-chaired by former "
                 "Surgeon General Vivek Murthy, former FDA commissioner David A. Kessler, "
                 "and Yale University's Marcella Nunez-Smith. On January 5, 2021, "
                 "the Democratic Party won control of the United States Senate, "
                 "effective January 20, as a result of electoral victories in "
                 "Georgia by Jon Ossoff in a runoff election for a six-year term "
                 "and Raphael Warnock in a special runoff election for a two-year term. "
                 "President-elect Biden had supported and campaigned for both "
                 "candidates prior to the runoff elections on January 5.On January 6, "
                 "a mob of thousands of Trump supporters violently stormed the Capitol "
                 "in the hope of overturning Biden's election, forcing Congress to "
                 "evacuate during the counting of the Electoral College votes. More "
                 "than 26,000 National Guard members were deployed to the capital "
                 "for the inauguration, with thousands remaining into the spring."},

        {"query": "What is the list of the top financial highlights for the quarter?",
         "answer": "•Revenue: $52.9 million, up 10% in constant currency;\n"
                   "•Operating income: $22.4 billion, up 15% in constant currency;\n"
                   "•Net income: $18.3 billion, up 14% in constant currency;\n"
                   "•Diluted earnings per share: $2.45 billion, up 14% in constant currency.",
         "context": "Microsoft Cloud Strength Drives Third Quarter Results \nREDMOND, Wash. — April 25, 2023 — "
                    "Microsoft Corp. today announced the following results for the quarter ended March 31, 2023,"
                    " as compared to the corresponding period of last fiscal year:\n· Revenue was $52.9 billion"
                    " and increased 7% (up 10% in constant currency)\n· Operating income was $22.4 billion "
                    "and increased 10% (up 15% in constant currency)\n· Net income was $18.3 billion and "
                    "increased 9% (up 14% in constant currency)\n· Diluted earnings per share was $2.45 "
                    "and increased 10% (up 14% in constant currency).\n"},

    {"query": "What is a list of the key points?",
     "answer": "•Stocks rallied on Friday with stronger-than-expected U.S jobs data and increase in "
               "Treasury yields;\n•Dow Jones gained 195.12 points;\n•S&P 500 added 1.59%;\n•Nasdaq Composite rose "
               "1.35%;\n•U.S. economy added 438,000 jobs in August, better than the 273,000 expected;\n"
               "•10-year Treasury rate trading near the highest level in 14 years at 4.58%.",
     "context": "Stocks rallied Friday even after the release of stronger-than-expected U.S. jobs data "
               "and a major increase in Treasury yields.  The Dow Jones Industrial Average gained 195.12 points, "
               "or 0.76%, to close at 31,419.58. The S&P 500 added 1.59% at 4,008.50. The tech-heavy "
               "Nasdaq Composite rose 1.35%, closing at 12,299.68. The U.S. economy added 438,000 jobs in "
               "August, the Labor Department said. Economists polled by Dow Jones expected 273,000 "
               "jobs. However, wages rose less than expected last month.  Stocks posted a stunning "
               "turnaround on Friday, after initially falling on the stronger-than-expected jobs report. "
               "At its session low, the Dow had fallen as much as 198 points; it surged by more than "
               "500 points at the height of the rally. The Nasdaq and the S&P 500 slid by 0.8% during "
               "their lowest points in the day.  Traders were unclear of the reason for the intraday "
               "reversal. Some noted it could be the softer wage number in the jobs report that made "
               "investors rethink their earlier bearish stance. Others noted the pullback in yields from "
               "the day’s highs. Part of the rally may just be to do a market that had gotten extremely "
               "oversold with the S&P 500 at one point this week down more than 9% from its high earlier "
               "this year.  Yields initially surged after the report, with the 10-year Treasury rate trading "
               "near its highest level in 14 years. The benchmark rate later eased from those levels, but "
               "was still up around 6 basis points at 4.58%.  'We’re seeing a little bit of a give back "
               "in yields from where we were around 4.8%. [With] them pulling back a bit, I think that’s "
               "helping the stock market,' said Margaret Jones, chief investment officer at Vibrant Industries "
               "Capital Advisors. 'We’ve had a lot of weakness in the market in recent weeks, and potentially "
               "some oversold conditions.'"}

    ]

    return test_list


def fast_start_prompting  (model_name):

    """ This is the main example script - it loads the question list, loads the model and executes the prompts. """

    t0 = time.time()

    # load in the 'hello world' test questions above
    test_list = hello_world_questions()

    print(f"\n > Loading Model: {model_name}...")

    prompter = Prompt().load_model(model_name)

    t1 = time.time()
    print(f"\n > Model {model_name} load time: {t1-t0} seconds")

    for i, entries in enumerate(test_list):
        print(f"\n{i+1}. Query: {entries['query']}")

        #   run the prompt
        output = prompter.prompt_main(entries["query"],
                                      context=entries["context"],
                                      prompt_name="default_with_context",
                                      temperature=0.30)

        #   'output' is a dictionary with two keys - 'llm_response' and 'usage'
        #   --'llm_response' is the output from the model
        #   --'usage' is a dictionary with the usage stats

        llm_response = output["llm_response"].strip("\n")
        print(f"LLM Response: {llm_response}")

        #   note: the 'gold answer' is the answer we provided above in the hello_world question list
        print(f"Gold Answer: {entries['answer']}")

        print(f"LLM Usage: {output['usage']}")

    t2 = time.time()
    print(f"\nTotal processing time: {t2-t1} seconds")

    return 0


if __name__ == "__main__":

    #   Step 1 - we will pick a model from the ModelCatalog

    #   A few useful methods to discover and display a list of available models...

    #   all generative models
    llm_models = ModelCatalog().list_generative_models()

    #   if you only want to see the local models
    llm_local_models = ModelCatalog().list_generative_local_models()

    #   to see only the open source models
    llm_open_source_models = ModelCatalog().list_open_source_models()

    #   we will print out the local models
    for i, models in enumerate(llm_local_models):
        print("models: ", i, models["model_name"], models["model_family"])

    #   for purposes of demo, try a few selected models from the list

    model_short_list = ["llmware/bling-1b-0.1",
                        "llmware/bling-tiny-llama-v0",
                        "llmware/dragon-yi-6b-gguf",
                        "llmware/bling-falcon-1b-0.1"]

    model_name = "llmware/bling-1b-0.1"

    #   to swap in a gpt-4 openai model - uncomment these two lines
    #   model_name = "gpt-4"
    #   os.environ["USER_MANAGED_OPENAI_API_KEY"] = "<insert-your-openai-key>"

    fast_start_prompting(model_name)

models:  0 Meta-Llama-3-8B HFGenerativeModel
models:  1 Meta-Llama-3-8B-Instruct HFGenerativeModel
models:  2 QuantFactory/Meta-Llama-3-8B-GGUF GGUFGenerativeModel
models:  3 QuantFactory/Meta-Llama-3-8B-Instruct-GGUF GGUFGenerativeModel
models:  4 TheBloke/Llama-2-7B-Chat-GGUF GGUFGenerativeModel
models:  5 TheBloke/OpenHermes-2.5-Mistral-7B-GGUF GGUFGenerativeModel
models:  6 TheBloke/Starling-LM-7B-alpha-GGUF GGUFGenerativeModel
models:  7 TheBloke/zephyr-7B-beta-GGUF GGUFGenerativeModel
models:  8 bartowski/Meta-Llama-3-8B-Instruct-GGUF GGUFGenerativeModel
models:  9 bling-answer-tool GGUFGenerativeModel
models:  10 bling-phi-3-gguf GGUFGenerativeModel
models:  11 bling-stablelm-3b-tool GGUFGenerativeModel
models:  12 dragon-llama-answer-tool GGUFGenerativeModel
models:  13 dragon-mistral-answer-tool GGUFGenerativeModel
models:  14 dragon-yi-answer-tool GGUFGenerativeModel
models:  15 llmware/bling-1.4b-0.1 HFGenerativeModel
models:  16 llmware/bling-1b-0.1 HFGenerativeModel
models

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]


 > Model llmware/bling-1b-0.1 load time: 110.88845658302307 seconds

1. Query: What is the total amount of the invoice?
LLM Response:  $22,500.00
Gold Answer: $22,500.00
LLM Usage: {'input': 157, 'output': 7, 'total': 164, 'metric': 'tokens', 'processing_time': 10.258683919906616}

2. Query: What was the amount of the trade surplus?
LLM Response:  62.4 billion yen
Gold Answer: 62.4 billion yen ($416.6 million)
LLM Usage: {'input': 147, 'output': 7, 'total': 154, 'metric': 'tokens', 'processing_time': 7.840357542037964}

3. Query: What was Microsoft's revenue in the 3rd quarter?
LLM Response:  $52.9 billion and increased 7% (up 10% in constant currency)
Gold Answer: $52.9 billion
LLM Usage: {'input': 162, 'output': 18, 'total': 180, 'metric': 'tokens', 'processing_time': 13.084855794906616}

4. Query: When did the LISP machine market collapse?
LLM Response:  1987
Gold Answer: 1987.
LLM Usage: {'input': 368, 'output': 2, 'total': 370, 'metric': 'tokens', 'processing_time': 12.6712834835

In [3]:

"""     Fast Start Example #4 - RAG with Text Query

    This example shows a basic RAG recipe using text query combined with LLM prompt.

    We will show two different ways to achieve this basic recipe:

    -- Example 4A - this will integrate Library + Prompt - and is the most scalable general solution

    -- Example 4B - this will illustrate another capability of the Prompt class to add sources "inline"
     without necessarily a library in-place.  It is another useful tool when you want to be able to quickly
     pick up a document and start asking questions to it.

     Note: both of the examples are designed to achieve the same output.

"""

import os
import re
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.setup import Setup
from llmware.configs import LLMWareConfig
from llmware.retrieval import Query
from llmware.library import Library


def example_4a_contract_analysis_from_library (model_name, verbose=False):

    """ Example #4a:  Main general case to run a RAG workflow from a Library """

    # Load the llmware sample files
    print (f"\n > Loading the llmware sample files...")
    sample_files_path = Setup().load_sample_files()
    contracts_path = os.path.join(sample_files_path,"Agreements")

    contracts_lib = Library().create_new_library("example4_library")
    contracts_lib.add_files(contracts_path)

    # questions that we want to ask each contract
    question_list = [{"topic": "executive employment agreement", "llm_query": "What are the names of the two parties?"},
                     {"topic": "base salary", "llm_query": "What is the executive's base salary?"},
                     {"topic": "governing law", "llm_query": "What is the governing law?"}]

    print (f"\n > Loading model {model_name}...")

    q = Query(contracts_lib)

    # get a list of all of the unique documents in the library

    # doc id list
    doc_list = q.list_doc_id()
    print("update: document id list - ", doc_list)

    # filename list
    fn_list = q.list_doc_fn()
    print("update: filename list - ", fn_list)

    prompter = Prompt().load_model(model_name)

    for i, doc_id in enumerate(doc_list):

        print("\nAnalyzing contract: ", str(i+1), doc_id, fn_list[i])

        print("LLM Responses:")

        for question in question_list:

            query_topic = question["topic"]
            llm_question = question["llm_query"]

            doc_filter = {"doc_ID": [doc_id]}
            query_results = q.text_query_with_document_filter(query_topic,doc_filter,result_count=5,exact_mode=True)

            if verbose:
                # this will display the query results from the query above
                for j, qr in enumerate(query_results):
                    print("update: querying document - ", query_topic, j, doc_filter, qr)

            source = prompter.add_source_query_results(query_results)

            #   *** this is the call to the llm with the source packaged in the context automatically ***
            responses = prompter.prompt_with_source(llm_question, prompt_name="default_with_context", temperature=0.3)

            #   unpacking the results from the LLM
            for r, response in enumerate(responses):
                print("update: llm response -  ", llm_question, re.sub("[\n]"," ", response["llm_response"]).strip())

            # We're done with this contract, clear the source from the prompt
            prompter.clear_source_materials()

    #   Save jsonl report to jsonl to /prompt_history folder
    print("\nPrompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(),prompter.prompt_id))
    prompter.save_state()

    #   Save csv report that includes the model, response, prompt, and evidence for human-in-the-loop review
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()
    print("\nCSV output saved at:  ", csv_output)

    return 0


def example_4b_contract_analysis_direct_from_prompt(model_name, verbose=False):

    """ Example #4b: Alternative implementation using prompt in-line capabilities without using a library """

    # Load the llmware sample files
    print(f"\n > Loading the llmware sample files...")
    sample_files_path = Setup().load_sample_files()
    contracts_path = os.path.join(sample_files_path, "Agreements")

    # questions that we want to ask each contract
    question_list = [{"topic": "executive employment agreement", "llm_query": "What are the names of the two parties?"},
                     {"topic": "base salary", "llm_query": "What is the executive's base salary?"},
                     {"topic": "governing law", "llm_query": "What is the governing law?"}]

    print(f"\n > Loading model {model_name}...")

    prompter = Prompt().load_model(model_name)

    for i, contract in enumerate(os.listdir(contracts_path)):

        # exclude potential mac os created file artifact in the samples folder path
        if contract != ".DS_Store":

            print("\nAnalyzing contract: ", str(i + 1), contract)

            print("LLM Responses:")

            for question in question_list:

                query_topic = question["topic"]
                llm_question = question["llm_query"]

                #   introducing "add_source_document"
                #   this will perform 'inline' parsing, text chunking and query filter on a document
                #   input is a file folder path, file name, and an optional query filter
                #   the source is automatically packaged into the prompt object

                source = prompter.add_source_document(contracts_path,contract,query=query_topic)

                if verbose:
                    print("update: document created source - ", source)

                #   calling the LLM with 'source' information from the contract automatically packaged into the prompt
                responses = prompter.prompt_with_source(llm_question, prompt_name="default_with_context",
                                                        temperature=0.3)

                #   unpacking the LLM responses
                for r, response in enumerate(responses):
                    print("update: llm response: ", llm_question, re.sub("[\n]", " ",
                                                                         response["llm_response"]).strip())

                # We're done with this contract, clear the source from the prompt
                prompter.clear_source_materials()

    # Save jsonl report to jsonl to /prompt_history folder
    print("\nupdate: Prompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(), prompter.prompt_id))
    prompter.save_state()

    # Save csv report that includes the model, response, prompt, and evidence for human-in-the-loop review
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()
    print("\nupdate: CSV output saved at - ", csv_output)

    return 0


if __name__ == "__main__":

    #   you can pick any model from the ModelCatalog
    #   we list a few representative good choices below

    LLMWareConfig().set_active_db("sqlite")

    example_models = ["llmware/bling-1b-0.1", "llmware/bling-tiny-llama-v0", "llmware/dragon-yi-6b-gguf"]

    #   to swap in a gpt-4 openai model - uncomment these two lines
    #   model_name = "gpt-4"
    #   os.environ["USER_MANAGED_OPENAI_API_KEY"] = "<insert-your-openai-key>"

    # use local cpu model
    model_name = example_models[0]

    #   two good recipes to address the use case

    #   first let's look at the main way of retrieving and analyzing from a library
    example_4a_contract_analysis_from_library(model_name)

    #   second - uncomment this line, and lets run the "in-line" prompt way
    # example_4b_contract_analysis_direct_from_prompt(model_name)


 > Loading the llmware sample files...

 > Loading model llmware/bling-1b-0.1...
update: document id list -  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
update: filename list -  ['Nike EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Bia EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Artemis Poseidon EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Athena EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Aphrodite EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Eileithyia EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Amphitrite EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Nyx EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Metis EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Rhea EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Demeter EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Persephone EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Apollo EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Leto EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Gaia EXECUTIVE EMPLOYMENT AGREEMENT.pdf']

Analyzing contract:  1 1 Nike EXECUTIVE EMPLOYMENT AGREEMENT.pdf
LLM Responses:
update: llm response -   What are the names o

In [5]:

"""     Fast Start Example #5 - RAG with Semantic Query

    This example illustrates the most common RAG retrieval pattern, which is using a semantic query, e.g.,
    a natural language query, as the basis for retrieving relevant text chunks, and then using as
    the context material in a prompt to ask the same question to a LLM.

    In this example, we will show the following:

    1.  Create library and install embeddings (feel free to skip / substitute a library created in an earlier step).
    2.  Ask a general semantic query to the entire library collection.
    3.  Select the most relevant results by document.
    4.  Loop through all of the documents - packaging the context and asking our questions to the LLM.

    NOTE: to use chromadb, you may need to install the python sdk:  pip3 install chromadb.

"""


import os
from llmware.library import Library
from llmware.retrieval import Query
from llmware.setup import Setup
from llmware.status import Status
from llmware.prompts import Prompt
from llmware.configs import LLMWareConfig
from importlib import util
if not util.find_spec("chromadb"):
    print("\nto run this example with chromadb, you need to install the chromadb python sdk:  pip3 install chromadb")


def semantic_rag (library_name, embedding_model_name, llm_model_name):

    """ Illustrates the use of semantic embedding vectors in a RAG workflow
        --self-contained example - will be duplicative with some of the steps taken in other examples """

    # Step 1 - Create library which is the main 'organizing construct' in llmware
    print ("\nupdate: Step 1 - Creating library: {}".format(library_name))

    library = Library().create_new_library(library_name)

    # Step 2 - Pull down the sample files from S3 through the .load_sample_files() command
    #   --note: if you need to refresh the sample files, set 'over_write=True'
    print ("update: Step 2 - Downloading Sample Files")

    sample_files_path = Setup().load_sample_files(over_write=False)
    contracts_path = os.path.join(sample_files_path, "Agreements")

    # Step 3 - point ".add_files" method to the folder of documents that was just created
    #   this method parses all of the documents, text chunks, and captures in MongoDB
    print("update: Step 3 - Parsing and Text Indexing Files")

    library.add_files(input_folder_path=contracts_path, chunk_size=400, max_chunk_size=600,
                      smart_chunking=1)

    # Step 4 - Install the embeddings
    print("\nupdate: Step 4 - Generating Embeddings in {} db - with Model- {}".format(vector_db, embedding_model))

    library.install_new_embedding(embedding_model_name=embedding_model_name, vector_db=vector_db)

    # RAG steps start here ...

    print("\nupdate: Loading model for LLM inference - ", llm_model_name)

    prompter = Prompt().load_model(llm_model_name)

    query = "what is the executive's base annual salary"

    #   key step: run semantic query against the library and get all of the top results
    results = Query(library).semantic_query(query, result_count=50, embedding_distance_threshold=1.0)

    #   if you want to look at 'results', uncomment the two lines below
    #   for i, res in enumerate(results):
    #       print("update: ", i, res["file_source"], res["distance"], res["text"])

    for i, contract in enumerate(os.listdir(contracts_path)):

        qr = []

        if contract != ".DS_Store":

            print("\nContract Name: ", i, contract)

            #   we will look through the list of semantic query results, and pull the top results for each file
            for j, entries in enumerate(results):

                library_fn = entries["file_source"]
                if os.sep in library_fn:
                    # handles difference in windows file formats vs. mac / linux
                    library_fn = library_fn.split(os.sep)[-1]

                if library_fn == contract:
                    print("Top Retrieval: ", j, entries["distance"], entries["text"])
                    qr.append(entries)

            #   we will add the query results to the prompt
            source = prompter.add_source_query_results(query_results=qr)

            #   run the prompt
            response = prompter.prompt_with_source(query, prompt_name="default_with_context", temperature=0.3)

            #   note: prompt_with_resource returns a list of dictionary responses
            #   -- depending upon the size of the source context, it may call the llm several times
            #   -- each dict entry represents 1 call to the LLM

            for resp in response:
                if "llm_response" in resp:
                    print("\nupdate: llm answer - ", resp["llm_response"])

            # start fresh for next document
            prompter.clear_source_materials()

    return 0


if __name__ == "__main__":

    LLMWareConfig().set_active_db("sqlite")

    #   for this example, we will use an embedding model that has been 'fine-tuned' for contracts
    embedding_model = "industry-bert-contracts"

    #   note: as of llmware==0.2.12, we have shifted from faiss to chromadb for the Fast Start examples
    #   --if you are using a Python version before 3.12, please feel free to substitute for "faiss"
    #   --for versions of Python >= 3.12, for the Fast Start examples (e.g., no install required), we
    #   recommend using chromadb or lancedb

    #   please double-check: `pip3 install chromadb` or pull the latest llmware version to get automatically
    #   -- if you have installed any other vector db, just change the name, e.g, "milvus" or "pg_vector"

    vector_db = "chromadb"

    # pick any name for the library
    lib_name = "example_5_library"

    example_models = ["llmware/bling-1b-0.1", "llmware/bling-tiny-llama-v0", "llmware/dragon-yi-6b-gguf"]

    # use local cpu model
    llm_model_name = example_models[0]

    #   to swap in a gpt-4 openai model - uncomment these two lines
    #   llm_model_name = "gpt-4"
    #   os.environ["USER_MANAGED_OPENAI_API_KEY"] = "<insert-your-openai-key>"

    semantic_rag(lib_name, embedding_model, llm_model_name)


update: Step 1 - Creating library: example_5_library
update: Step 2 - Downloading Sample Files
update: Step 3 - Parsing and Text Indexing Files

update: Step 4 - Generating Embeddings in chromadb db - with Model- industry-bert-contracts


config.json:   0%|          | 0.00/803 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

NameError: name 'chromadb' is not defined

In [6]:

"""     Fast Start Example #6 - RAG - Beyond the Basics

    This example builds upon examples #4 and #5 and demonstrates how to layer additional elements to
    improve the effectiveness of a RAG workflow over a larger set of documents-

    1.  Apply an initial filter across a batch of documents to identify a subset of documents of interest
    2.  Analyze the documents of interest to identify key provisions
    3.  Use fact-checking and post-processing to validate the accuracy of the LLM response
    4.  Write the output to JSON and CSV files for follow-up review and/or the next step in the workflow.

    For this example, we also recommend using a more sophisticated DRAGON model in GGUF format, which enables us
    to run 6-7B parameter models locally.

"""

import os

from llmware.setup import Setup
from llmware.library import Library
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.retrieval import Query
from llmware.configs import LLMWareConfig


def msa_processing(library_name, llm_model_name):

    """ In this example, we will use the 'AgreementsLarge' sample files which consists of ~80 contracts.  We
    need to quickly identify the 'master service agreements' as we only want to analyze those contracts. """

    local_path = Setup().load_sample_files()
    agreements_path = os.path.join(local_path, "AgreementsLarge")

    #   create a library with all of the Agreements (~80 contracts)
    print(f"\nStarting:  Parsing 'AgreementsLarge' Folder")
    msa_lib = Library().create_new_library(library_name)
    msa_lib.add_files(agreements_path)

    #   find the "master service agreements" (MSA) - we know that 'master services agreement' will always
    #   be on the first page of the agreement, so we can use that as a good proxy for automatically filtering
    #   to our target set of documents

    print(f"\nCompleted Parsing - now, let's look for the 'master service agreements', e.g., 'msa'")

    q = Query(msa_lib)
    query = '"master services agreement"'
    results = q.text_search_by_page(query, page_num=1, results_only=False)

    #   results_only = False will return a dictionary with 4 keys:  {"query", "results", "doc_ID", "file_source"}
    msa_docs = results["file_source"]
    msa_doc_ids = results["doc_ID"]

    #   load prompt/llm locally
    prompter = Prompt().load_model(llm_model_name)

    print("update: identified the following msa doc id: ", msa_doc_ids)

    #   analyze each MSA - "query" & "llm prompt"
    for i, doc_id in enumerate(msa_doc_ids):

        print("\n")
        docs = msa_docs[i]
        if os.sep in docs:
            # handles difference in windows file formats vs. Mac/Linux
            docs = docs.split(os.sep)[-1]

        print (i+1, "Reviewing MSA - ", doc_id, docs)

        #   look for the termination provisions in each document
        doc_filter = {"doc_ID": [doc_id]}
        termination_provisions = q.text_query_with_document_filter("termination", doc_filter)

        #   package the provisions as a source to a prompt
        sources = prompter.add_source_query_results(termination_provisions)

        #   if you want to see more details about how the sources are packaged: uncomment this line-
        #   print("update: sources - ", sources)

        #   call the LLM and ask our question
        response = prompter.prompt_with_source("What is the notice for termination for convenience?")

        #   post processing fact checking
        stats = prompter.evidence_comparison_stats(response)
        ev_source = prompter.evidence_check_sources(response)

        for i, resp in enumerate(response):
            print("update: llm response - ", resp)
            print("update: compare with evidence- ", stats[i]["comparison_stats"])
            print("update: sources - ", ev_source[i]["source_review"])

        prompter.clear_source_materials()

    # Save jsonl report with full transaction history to /prompt_history folder
    print("\nupdate: Prompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(),prompter.prompt_id))

    prompter.save_state()

    # Generate CSV report for easy Human review in Excel
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()

    print("\nupdate: CSV output for human review - ", csv_output)

    return 0


if __name__ == "__main__":

    LLMWareConfig().set_active_db("sqlite")

    #   this is part of the DRAGON model series - RAG-fine-tuned fact-based Q&A model
    llm = "llmware/dragon-yi-6b-gguf"

    m = msa_processing("example6_library", llm)


Starting:  Parsing 'AgreementsLarge' Folder

Completed Parsing - now, let's look for the 'master service agreements', e.g., 'msa'


dragon-yi-6b-q4_k_m.gguf:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

update: identified the following msa doc id:  [1, 7, 9, 14, 15, 26, 30, 31, 32, 36, 40, 41, 46, 47, 49, 52, 57, 60, 65, 67]


1 Reviewing MSA -  1 Testco MSA Sigma.pdf
update: llm response -  {'llm_response': '30 days written notice.', 'prompt': 'What is the notice for termination for convenience?', 'evidence': " party  may  be   deemed  notice  of   termination  of  this  Agreement,  effective  on  the  date  of  assignment, by the other party.   Survival upon Termination or End of Term.   The   provisions  regarding  warranty,  limitation  of  liability,  confidentiality,  fees  and   expenses,  obligations  on   termination or expiration, ownership and license, and miscellaneous of this  Agreement, and \n or  any  work  order  or  services  description  if  the  other  party  is  in  material  breach  or   default  of any obligation that is not cured  within 15 days' notice of such breach. The TestCo agrees to pay all fees   for services performed  and  expenses  incurred  prior  to

In [ ]:
import os
from llmware.library import Library
from llmware.retrieval import Query
from llmware.setup import Setup
from llmware.configs import LLMWareConfig

In [ ]:
def parsing_documents_into_library(library_name, sample_folder):
  print(f"\nExample - Parsing Files into Library")
  #create new library
  print(f"Creating new library {library_name}")
  library = Library().create_new_library(library_name)

  #load the llmware sample files

  sample_files_path = Setup().load_sample_files(over_write=False)
  print(f"Loading llmware  files from {sample_files_path}")
    #   note: to replace with your own documents, just point to a local folder path that has the documents
  ingestion_folder_path = os.path.join(sample_files_path, sample_folder)

  print (f"Step 3 - parsing and indexing files from {ingestion_folder_path}")
    #   add files is the key ingestion method - parses, text chunks and indexes all files in folder
    #       --will automatically route to correct parser based on file extension
    #       --supported file extensions:  .pdf, .pptx, .docx, .xlsx, .csv, .md, .txt, .json, .wav, and .zip, .jpg, .png

  parsing_output = library.add_files(ingestion_folder_path)

  print (f"Step 4 - completed parsing - {parsing_output}")

    #   check the updated library card
  updated_library_card = library.get_library_card()
  doc_count = updated_library_card["documents"]
  block_count = updated_library_card["blocks"]
  print(f"Step 5 - updated library card - documents - {doc_count} - blocks - {block_count} - {updated_library_card}")

    #   check the main folder structure created for the library - check /images to find extracted images
  library_path = library.library_main_path
  print(f"Step 6 - library artifacts - including extracted images - saved at folder path - {library_path}")

    #   use .add_files as many times as needed to build up your library, and/or create different libraries for
    #   different knowledge bases

    #   now, your library is ready to go and you can start to use the library for running queries
    #   if you are using the "Agreements" library, then a good easy 'hello world' query is "base salary"
    #   if you are using one of the other sample folders (or your own), then you should adjust the query

    #   queries are always created the same way, e.g., instantiate a Query object, and pass a library object
    #   --within the Query class, there are a variety of useful methods to run different types of queries

  test_query = "base salary"

  print(f"\nStep 7 - running a test query - {test_query}\n")

  query_results = Query(library).text_query(test_query, result_count=10)

  for i, result in enumerate(query_results):

        #   note: each result is a dictionary with a wide range of useful keys
        #   -- we would encourage you to take some time to review each of the keys and the type of metadata available

        #   here are a few useful attributes
      text = result["text"]
      file_source = result["file_source"]
      page_number = result["page_num"]
      doc_id = result["doc_ID"]
      block_id = result["block_ID"]
      matches = result["matches"]

        #   -- if print to console is too verbose, then pick just a few keys for print
      print("query results: ", i, result)

  return parsing_output


if __name__ == "__main__":

    #  note on sample documents - downloaded by Setup()
    #       UN-Resolutions-500 is 500 pdf documents
    #       Invoices is 40 pdf invoice samples
    #       Agreements is ~15 contract documents
    #       AgreementsLarge is ~80 contract documents
    #       FinDocs is ~15 financial annual reports and earnings
    #       SmallLibrary is a mix of ~10 pdf and office documents

    #   optional - set the active DB to be used - by default, it is "mongo"
    #   if you are just getting started, and have not installed a separate db, select "sqlite"

    LLMWareConfig().set_active_db("sqlite")

    #   if you want to see a different log view, e.g., see a list of each parsed files 'in progress',
    #   you can set a different debug mode view anytime

    #   debug_mode options -
    #       0 - default - shows status manager (useful in large parsing jobs) and errors will be displayed
    #       2 - file name only - shows file name being parsed, and errors only

    #   for purpose of this example, let's change so we can see file-by-file progress
    LLMWareConfig().set_config("debug_mode", 2)

    #   this is a list of document folders that will be pulled down by calling Setup()
    sample_folders = ["Invoices", "UN-Resolutions-500", "SmallLibrary", "FinDocs", "AgreementsLarge"]

    library_name = "example1_library"

    #   select one of the sample folders
    selected_folder = sample_folders[1]     # e.g., "Agreements"

    #   run the example
    output = parsing_documents_into_library(library_name, selected_folder)




Example - Parsing Files into Library
Creating new library example1_library
Loading llmware  files from /root/llmware_data/sample_files
Step 3 - parsing and indexing files from /root/llmware_data/sample_files/UN-Resolutions-500
Step 4 - completed parsing - {'docs_added': 500, 'blocks_added': 24294, 'images_added': 1500, 'pages_added': 2701, 'tables_added': 2, 'rejected_files': []}
Step 5 - updated library card - documents - 1000 - blocks - 48588 - {'_id': 2, 'library_name': 'example1_library', 'embedding': [{'embedding_status': 'no', 'embedding_model': 'none', 'embedding_db': 'none', 'embedded_blocks': 0, 'embedding_dims': 0, 'time_stamp': 'NA'}], 'knowledge_graph': 'no', 'unique_doc_id': 1000, 'documents': 1000, 'blocks': 48588, 'images': 3000, 'pages': 5402, 'tables': 4, 'account_name': 'llmware'}
Step 6 - library artifacts - including extracted images - saved at folder path - /root/llmware_data/accounts/llmware/example1_library

Step 7 - running a test query - base salary

query res

In [4]:
!pip3 install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4

In [ ]:
""""    Fast Start Example #2 - Embeddings - applying an embedding model to enable natural language queries

    In this example, we will show the basic recipe for creating embeddings on a library:

    1.  Create a sample library (see Example #1 for more details)
    2   Select an embedding model
    3.  Select a vector db
    4.  Install the embeddings
    5.  Run a semantic test query

    For purpose of this 'fast start', we will use a no-install option of 'chromadb' and 'sqlite'

    Note: we have updated the no-install vector db option to 'chromadb' from 'faiss' starting in
    llmware>=0.2.12, due to better support on Python 3.12

    Note: you may need to install chromadb's python driver:  `pip3 install chromadb`

    -- This same basic recipe will work with any of the vector db and collection db by simply changing the name

"""


import os
from llmware.library import Library
from llmware.retrieval import Query
from llmware.setup import Setup
from llmware.status import Status
from llmware.models import ModelCatalog
from llmware.configs import LLMWareConfig
import chromadb
from importlib import util
if not util.find_spec("chromadb"):
    print("\nto run this example with chromadb, you need to install the chromadb python sdk:  pip3 install chromadb")


def setup_library(library_name):

    """ Note: this setup_library method is provided to enable a self-contained example to create a test library """

    #   Step 1 - Create library which is the main 'organizing construct' in llmware
    print ("\nupdate: Creating library: {}".format(library_name))

    library = Library().create_new_library(library_name)

    #   check the embedding status 'before' installing the embedding
    embedding_record = library.get_embedding_status()
    print("embedding record - before embedding ", embedding_record)

    #   Step 2 - Pull down the sample files from S3 through the .load_sample_files() command
    #   --note: if you need to refresh the sample files, set 'over_write=True'
    print ("update: Downloading Sample Files")

    sample_files_path = Setup().load_sample_files(over_write=False)

    #   Step 3 - point ".add_files" method to the folder of documents that was just created
    #   this method parses the documents, text chunks, and captures in database

    print("update: Parsing and Text Indexing Files")

    library.add_files(input_folder_path=os.path.join(sample_files_path, "Agreements"),
                      chunk_size=400, max_chunk_size=600, smart_chunking=1)

    return library


def install_vector_embeddings(library, embedding_model_name):

    """ This method is the core example of installing an embedding on a library.
        -- two inputs - (1) a pre-created library object and (2) the name of an embedding model """

    library_name = library.library_name
    vector_db = LLMWareConfig().get_vector_db()
    if vector_db != "chromadb":
        raise ValueError(f"Unexpected vector_db value: {vector_db}")

    print(f"\nupdate: Starting the Embedding: "
          f"library - {library_name} - "
          f"vector_db - {vector_db} - "
          f"model - {embedding_model_name}")

    #   *** this is the one key line of code to create the embedding ***
    library.install_new_embedding(embedding_model_name=embedding_model, vector_db=vector_db,batch_size=100)

    #   note: for using llmware as part of a larger application, you can check the real-time status by polling Status()
    #   --both the EmbeddingHandler and Parsers write to Status() at intervals while processing
    update = Status().get_embedding_status(library_name, embedding_model)
    print("update: Embeddings Complete - Status() check at end of embedding - ", update)

    # Start using the new vector embeddings with Query
    sample_query = "incentive compensation"
    print("\n\nupdate: Run a sample semantic/vector query: {}".format(sample_query))

    #   queries are constructed by creating a Query object, and passing a library as input
    query_results = Query(library).semantic_query(sample_query, result_count=20)

    for i, entries in enumerate(query_results):

        #   each query result is a dictionary with many useful keys

        text = entries["text"]
        document_source = entries["file_source"]
        page_num = entries["page_num"]
        vector_distance = entries["distance"]

        #   to see all of the dictionary keys returned, uncomment the line below
        #   print("update: query_results - all - ", i, entries)

        #  for display purposes only, we will only show the first 125 characters of the text
        if len(text) > 125:  text = text[0:125] + " ... "

        print("\nupdate: query results - {} - document - {} - page num - {} - distance - {} "
              .format( i, document_source, page_num, vector_distance))

        print("update: text sample - ", text)

    #   lets take a look at the library embedding status again at the end to confirm embeddings were created
    embedding_record = library.get_embedding_status()

    print("\nupdate:  embedding record - ", embedding_record)

    return 0


if __name__ == "__main__":

    #   Fast Start configuration - will use no-install embedded sqlite
    #   -- if you have installed Mongo or Postgres, then change the .set_active_db accordingly

    LLMWareConfig().set_active_db("sqlite")

    #   note: as of llmware==0.2.12, we have shifted from faiss to chromadb for the Fast Start examples
    #   --if you are using a Python version before 3.12, please feel free to substitute for "faiss"
    #   --for versions of Python >= 3.12, for the Fast Start examples (e.g., no install required), we
    #   recommend using chromadb or lancedb
    #   please double-check: `pip3 install chromadb` or pull the latest llmware version to get automatically

    #   -- if you have installed any other vector db, just change the name, e.g, "milvus" or "pg_vector"

    LLMWareConfig().set_vector_db("chromadb")

    #  Step 1 - this example requires us to have a library created - two options:

    #  if you completed example-1 - then load the library you created in that example, e.g., "example1_library"
    #  uncomment the line below:
    #  library = Library().load_library("example1_library")

    #  alternatively, to use this example as self-contained, then create a new library from scratch:
    library = setup_library("example2_library")

    #   Step 2 - Select any embedding model in the LLMWare catalog

    #   to see a list of the embedding models supported, uncomment the line below and print the list
    embedding_models = ModelCatalog().list_embedding_models()

    #   for i, models in enumerate(embedding_models):
    #       print("embedding models: ", i, models)

    #   for this first embedding, we will use a very popular and fast sentence transformer
    embedding_model = "mini-lm-sbert"

    #   note: if you want to swap out "mini-lm-sbert" for Open AI 'text-embedding-ada-002', uncomment these lines:
    #   embedding_model = "text-embedding-ada-002"
    #   os.environ["USER_MANAGED_OPENAI_API_KEY"] = "<insert-your-openai-api-key>"

    #   run the core script
    install_vector_embeddings(library, embedding_model)


update: Creating library: example2_library
embedding record - before embedding  [{'embedding_status': 'no', 'embedding_model': 'none', 'embedding_db': 'none', 'embedded_blocks': 0, 'embedding_dims': 0, 'time_stamp': 'NA'}]
update: Downloading Sample Files
update: Parsing and Text Indexing Files

update: Starting the Embedding: library - example2_library - vector_db - chromadb - model - mini-lm-sbert


NameError: name 'chromadb' is not defined

In [ ]:
from chromadb import PersistentClient

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.0 MB/s eta 0:00:00


In [ ]:
!python -V

Python 3.10.12


In [ ]:
import sys

for path in sys.path:
    if "chromadb" in path:
        print("chromadb module found in:", path)

In [ ]:
pip show chromadb

Name: chromadb
Version: 0.5.0
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [ ]:
!python -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!python -m pip install chromadb

In [ ]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 12.6 MB/s eta 0:00:00


In [ ]:
!python -m virtualenv venv

created virtual environment CPython3.10.12.final.0-64 in 2037ms
  creator CPython3Posix(dest=/content/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source venv/bin/activate

In [ ]:
!pip install chromadb

In [ ]:
!pip show chromadb

Name: chromadb
Version: 0.5.0
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 
